# Voy a intentar normalizar work year y remote ratio e intentar hacer label encoder en las demas a ver que sale

In [1]:
import pandas as pd
import pylab as plt
import seaborn as sns
import numpy as np

In [2]:
from sklearn.model_selection import train_test_split as tts   
from sklearn.metrics import mean_squared_error as mse   
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor as RFR  
from sklearn.tree import ExtraTreeRegressor as ETR
from sklearn.ensemble import GradientBoostingRegressor as GBR
from xgboost import XGBRegressor as XGBR
from catboost import CatBoostRegressor as CTR
from sklearn.linear_model import LinearRegression as LinReg
from sklearn.linear_model import Lasso    
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet 


In [3]:
muestra=pd.read_csv('../data/muestra.csv')


In [4]:
muestra.head()

,id,salary_in_usd
0,0,70377
1,1,85839
2,2,63613
3,3,88561
4,4,37621


In [5]:
salaries = pd.read_csv('../data/salaries_data.csv')

In [6]:
salaries.head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2022,SE,FT,Data Engineer,140250,USD,140250,US,100,US,M
1,2022,SE,FT,Data Engineer,135000,USD,135000,US,100,US,M
2,2021,MI,FT,BI Data Analyst,100000,USD,100000,US,100,US,M
3,2021,MI,CT,ML Engineer,270000,USD,270000,US,100,US,L
4,2021,MI,FT,Data Engineer,22000,EUR,26005,RO,0,US,L


In [7]:
testeo = pd.read_csv('../data/testeo.csv')

In [8]:
testeo.head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,employee_residence,remote_ratio,company_location,company_size
0,2020,SE,FT,Machine Learning Scientist,260000,USD,JP,0,JP,S
1,2020,MI,FT,Lead Data Analyst,87000,USD,US,100,US,L
2,2020,MI,FT,Data Analyst,85000,USD,US,100,US,L
3,2020,MI,FT,Machine Learning Engineer,299000,CNY,CN,0,CN,M
4,2020,MI,FT,Product Data Analyst,450000,INR,IN,100,IN,L


In [9]:
salaries.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   work_year           500 non-null    int64 
 1   experience_level    500 non-null    object
 2   employment_type     500 non-null    object
 3   job_title           500 non-null    object
 4   salary              500 non-null    int64 
 5   salary_currency     500 non-null    object
 6   salary_in_usd       500 non-null    int64 
 7   employee_residence  500 non-null    object
 8   remote_ratio        500 non-null    int64 
 9   company_location    500 non-null    object
 10  company_size        500 non-null    object
dtypes: int64(4), object(7)
memory usage: 224.3 KB


In [10]:
salaries.shape

(500, 11)

In [11]:
testeo.shape

(107, 10)

In [12]:
salaries2 = salaries.copy()


In [13]:
stats=salaries2.describe().T
stats['IQR']=stats['75%']-stats['25%']
outliers=pd.DataFrame(columns=stats.index)
for c in stats.index:
    iqr=stats.at[c, 'IQR']
    cutoff=1.5 * iqr   # test de tukey
    lower=stats.at[c, '25%'] - cutoff
    upper=stats.at[c, '75%'] + cutoff
    res=salaries2[(salaries2[c] < lower) | (salaries2[c] > upper)].copy()
    res['outliers']=c
    outliers=outliers.append(res, sort=True)
outliers['outliers'].value_counts()
clean_index=[i for i in salaries2.index if i not in outliers.index]
salaries2=salaries2.iloc[clean_index]

/var/folders/0p/q9p3n_m96v7fgw9v0l4dvkyr0000gn/T/ipykernel_26435/1326823474.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  outliers=outliers.append(res, sort=True)
/var/folders/0p/q9p3n_m96v7fgw9v0l4dvkyr0000gn/T/ipykernel_26435/1326823474.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  outliers=outliers.append(res, sort=True)
/var/folders/0p/q9p3n_m96v7fgw9v0l4dvkyr0000gn/T/ipykernel_26435/1326823474.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  outliers=outliers.append(res, sort=True)
/var/folders/0p/q9p3n_m96v7fgw9v0l4dvkyr0000gn/T/ipykernel_26435/1326823474.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  outli

In [14]:
y=salaries2.salary_in_usd

In [15]:
salaries2.drop('salary_in_usd',axis=1,inplace=True)

In [16]:
#salaries2.drop(['salary','salary_currency'],axis=1,inplace=True)

In [17]:
salaries2.head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,employee_residence,remote_ratio,company_location,company_size
0,2022,SE,FT,Data Engineer,140250,USD,US,100,US,M
1,2022,SE,FT,Data Engineer,135000,USD,US,100,US,M
2,2021,MI,FT,BI Data Analyst,100000,USD,US,100,US,M
3,2021,MI,CT,ML Engineer,270000,USD,US,100,US,L
4,2021,MI,FT,Data Engineer,22000,EUR,RO,0,US,L


In [18]:
salaries2.shape

(463, 10)

In [19]:
salaries3 = pd.concat([salaries2, testeo]).reset_index()

In [20]:
salaries3.drop('index',axis=1,inplace=True)

In [21]:
salaries3.shape

(570, 10)

# Hasta aqui bien, ahora a ver como limpio...

In [ ]:
plt.figure(figsize=(15, 10))

sns.set(style='white')

mask=np.triu(np.ones_like(salaries3.corr(), dtype=bool))

cmap=sns.diverging_palette(0, 10, as_cmap=True)


sns.heatmap(salaries.corr(),
          cmap=cmap,
          center=0,
          square=True,
          annot=True,
          linewidths=0.5,
          cbar_kws={'shrink': 0.5});

In [ ]:
_ = salaries3.hist(figsize=(20, 14))

In [ ]:
salaries3["experience_level"].value_counts()

In [ ]:
salaries3["employment_type"].value_counts()

In [ ]:
jobs = salaries3["job_title"].value_counts()
jobs

In [ ]:
salaries3["employee_residence"].value_counts()

In [ ]:
salaries3["company_size"].value_counts()

In [22]:
salaries3.head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,employee_residence,remote_ratio,company_location,company_size
0,2022,SE,FT,Data Engineer,140250,USD,US,100,US,M
1,2022,SE,FT,Data Engineer,135000,USD,US,100,US,M
2,2021,MI,FT,BI Data Analyst,100000,USD,US,100,US,M
3,2021,MI,CT,ML Engineer,270000,USD,US,100,US,L
4,2021,MI,FT,Data Engineer,22000,EUR,RO,0,US,L


In [23]:
salaries3.drop(['salary','salary_currency'],axis=1,inplace=True)

In [ ]:
salaries3.head()

In [ ]:
super_high = ['US','NZ','CA','CA','AE','CN','DZ','SG','DK','JP','AU','BE']
high = ['DE','GB','AT','PL','IE','CZ','FR','CH','SI','RO','NL']
medium=['GR','ES','PT','LU','CL','HU','EE','IN','NG','MT']

In [ ]:
new_location = []
for e in salaries3.company_location:
    if e in super_high:
        new_location.append('super_high')
    elif e in high:
        new_location.append('high')
    elif e in medium:
        new_location.append('medium')
    else:
        new_location.append('low')

In [ ]:
salaries3.company_location = new_location

In [ ]:
salaries3.head()

In [31]:
salaries3.drop('employee_residence',axis=1, inplace=True)

In [ ]:
salaries3.head()

In [ ]:
def limpiar_job(column):
    if 'Scientist' in column:
        return 'Data Scientist'
    elif 'Engineer' in column:
        return 'Data Engineer'
    elif 'Analyst' in column:
        return 'Data Analyst '
    else:
        return 'Machine Learning Engineer'

In [ ]:
salaries3['job_title'] = salaries3['job_title'].apply(limpiar_job)

In [ ]:
salaries3.head()

# Hasta aqui creo que todo esta mas o menos bien
## Voy a probar label encoder a ver que pasa y despues puedo probar get dummies a ver que da mejor

In [ ]:
salaries3=pd.get_dummies(salaries3, columns=['experience_level', 'employment_type', 'job_title',\
                                         'company_location', 'company_size'], drop_first=True)

In [ ]:
salaries3.company_size.value_counts()

In [25]:
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder


In [26]:
salaries3.company_location=LabelEncoder().fit_transform(salaries3.company_location)

salaries3.head()

,work_year,experience_level,employment_type,job_title,employee_residence,remote_ratio,company_location,company_size
0,2022,SE,FT,Data Engineer,US,100,45,M
1,2022,SE,FT,Data Engineer,US,100,45,M
2,2021,MI,FT,BI Data Analyst,US,100,45,M
3,2021,MI,CT,ML Engineer,US,100,45,L
4,2021,MI,FT,Data Engineer,RO,0,45,L


In [27]:
salaries3.job_title=LabelEncoder().fit_transform(salaries3.job_title)

#salaries3.head()

In [28]:
salaries3.employment_type=LabelEncoder().fit_transform(salaries3.employment_type)


In [29]:
salaries3.experience_level=LabelEncoder().fit_transform(salaries3.experience_level)


In [33]:
salaries3.company_size=LabelEncoder().fit_transform(salaries3.company_size)


In [ ]:
salaries3=pd.get_dummies(salaries3, columns=['company_size','company_location','job_title','employment_type','experience_level'], drop_first=True)

salaries3.head()

# Normalizo workyear y remote ratio para ver que sale 

In [35]:
from sklearn.preprocessing import StandardScaler

scaler=StandardScaler()

In [36]:
#salaries3[['work_year','remote_ratio']] = scaler.fit_transform(salaries3[['work_year','remote_ratio']])


In [37]:
salaries3[['work_year','remote_ratio']] = scaler.fit_transform(salaries3[['work_year','remote_ratio']])


In [40]:
salaries3[['experience_level','employment_type','job_title','company_location','company_size']] = scaler.fit_transform(salaries3[['experience_level','employment_type','job_title','company_location','company_size']])


In [41]:
salaries3.shape

(570, 7)

In [42]:
salaries3.head()

,work_year,experience_level,employment_type,job_title,remote_ratio,company_location,company_size
0,0.820950,0.820465,0.023340,-0.406447,0.700449,0.731468,0.285361
1,0.820950,0.820465,0.023340,-0.406447,0.700449,0.731468,0.285361
2,-0.636812,-0.162025,0.023340,-1.519781,0.700449,0.731468,0.285361
3,-0.636812,-0.162025,-8.845906,1.415372,0.700449,0.731468,-1.278638
4,-0.636812,-0.162025,0.023340,-0.406447,-1.756510,0.731468,-1.278638


# Hasta aqui la limpieza ?¿ ahora a probar modelos

In [43]:
salaries4 = salaries3.iloc[0:463]
testeofin= salaries3.iloc[463:]

In [44]:
salaries4 = salaries3.iloc[:463,:]
testeofin= salaries3.iloc[463:,:]

In [45]:
salaries4.shape

(463, 7)

In [46]:
testeofin.shape

(107, 7)

In [47]:
y.shape

(463,)

In [48]:
x_train, x_test, y_train, y_test = tts(salaries4, y, train_size=0.8, test_size=0.2, random_state=22)

In [51]:
def predecir(x_train, x_test, y_train, y_test):
    #inicializamos todos los modelos que vamos a probar
    svr=SVR()
    rfr=RFR()
    etr=ETR()
    gbr=GBR()
    xgbr=XGBR()
    ctr=CTR()
    linreg=LinReg()
    lasso=Lasso()
    ridge=Ridge()
    elastic=ElasticNet()
    #los entrenamos
    svr.fit(x_train, y_train)
    rfr.fit(x_train, y_train)
    etr.fit(x_train, y_train)
    gbr.fit(x_train, y_train)
    xgbr.fit(x_train, y_train)
    ctr.fit(x_train, y_train, verbose=0)
    linreg.fit(x_train, y_train)
    lasso.fit(x_train, y_train)
    ridge.fit(x_train, y_train)
    elastic.fit(x_train, y_train)
    #predecimos
    y_pred1 = svr.predict(x_test)
    y_pred2 = rfr.predict(x_test)
    y_pred3 = etr.predict(x_test)
    y_pred4 = gbr.predict(x_test)
    y_pred5 = xgbr.predict(x_test)
    y_pred6 = ctr.predict(x_test)
    y_pred7 = linreg.predict(x_test)
    y_pred8 = lasso.predict(x_test)
    y_pred9 = ridge.predict(x_test)
    y_pred10 = elastic.predict(x_test)
    #calculamos error cuadrático medio (mse)
    mse1 = mse(y_test, y_pred1, squared=False)
    mse2 = mse(y_test, y_pred2, squared=False)
    mse3 = mse(y_test, y_pred3, squared=False)
    mse4 = mse(y_test, y_pred4, squared=False)
    mse5 = mse(y_test, y_pred5, squared=False)
    mse6 = mse(y_test, y_pred6, squared=False)
    mse7 = mse(y_test, y_pred7, squared=False)
    mse8 = mse(y_test, y_pred8, squared=False)
    mse9 = mse(y_test, y_pred9, squared=False)
    mse10 = mse(y_test, y_pred10, squared=False)
    #creamos una lista con todos los mse
    temp = [mse1, mse2, mse3, mse4, mse5, mse6, mse7, mse8, mse9, mse10]
    #pedimos a la función que nos devuelva el valor más bajo de mse
    minimo = min(temp)
    #le ponemos un mensajito para que quede más mono
    if minimo == mse1:
        return f'El mejor modelo es SVR con un mse de {mse1}'
    elif minimo == mse2:
        return f'El mejor modelo es RFR con un mse de {mse2}'
    elif minimo == mse3:
        return f'El mejor modelo es ETR con un mse de {mse3}'
    elif minimo == mse4:
        return f'El mejor modelo es GBR con un mse de {mse4}'
    elif minimo == mse5:
        return f'El mejor modelo es XGBR con un mse de {mse5}'
    elif minimo == mse6:
        return f'El mejor modelo es CTR con un mse de {mse6}'
    elif minimo == mse7:
        return f'El mejor modelo es LINREG con un mse de {mse7}'
    elif minimo == mse8:
        return f'El mejor modelo es LASSO con un mse de {mse8}'
    elif minimo == mse9:
        return f'El mejor modelo es RIDGE con un mse de {mse9}'
    elif minimo == mse10:
        return f'El mejor modelo es ELASTIC con un mse de {mse10}'

In [52]:
predecir(x_train, x_test, y_train, y_test)

'El mejor modelo es GBR con un mse de 33049.05472491613'

In [53]:
gbr=GBR()


In [54]:
gbr.fit(x_train, y_train)


GradientBoostingRegressor()

In [55]:
y_pred18 = gbr.predict(x_test)


In [56]:
mse17 = mse(y_test, y_pred18, squared=False)


In [57]:
mse17

33088.6718619581

In [60]:
y_pred_test15 = gbr.predict(testeofin)

In [61]:
muestra.salary_in_usd = y_pred_test15

In [62]:
error15 = muestra.copy()

In [63]:
error15.to_csv('../data/error15.csv', index=False)


# De aqui para abajo noooooo

In [ ]:
salaries.groupby('experience_level').mean()

In [ ]:
salaries3.describe(include='all').T

In [ ]:
salaries3=pd.get_dummies(salaries3, columns=['company_size'], drop_first=True)

salaries3.head()

In [ ]:
salaries3=pd.get_dummies(salaries3, columns=['experience_level'], drop_first=True)


In [ ]:
salaries3.job_title.value_counts()

In [ ]:
def limpiar_job(column):
    if 'Scientist' in column:
        return 'Data Scientist'
    elif 'Engineer' in column:
        return 'Data Engineer'
    elif 'Analyst' in column:
        return 'Data Analyst '
    else:
        return 'Machine Learning Engineer'

In [ ]:
salaries3['job_title'] = salaries3['job_title'].apply(limpiar_job)

In [ ]:
salaries3.head()

In [ ]:
salaries3.job_title.value_counts()

In [ ]:
salaries3=pd.get_dummies(salaries3, columns=['job_title'], drop_first=True)

salaries3.head()

In [ ]:
salaries3.employment_type.value_counts()

In [ ]:
salaries3=pd.get_dummies(salaries3, columns=['employment_type'], drop_first=True)

salaries3.head()

In [ ]:
#salaries3.drop('employee_residence',axis=1,inplace=True)

In [ ]:
salaries3["company_location"].value_counts()

In [ ]:
salaries3=pd.get_dummies(salaries3, columns=['employee_residence'], drop_first=True)

salaries3.head()

In [ ]:
salaries3=pd.get_dummies(salaries3, columns=['company_location'], drop_first=True)

salaries3.head()

In [ ]:
salaries3=pd.get_dummies(salaries3, columns=['remote_ratio'], drop_first=True)

salaries3.head()

In [ ]:
salaries3=pd.get_dummies(salaries3, columns=['work_year'], drop_first=True)

salaries3.head()

In [ ]:
prueba = salaries.groupby('company_location').mean()
prueba = prueba.reset_index()
prueba

In [ ]:
sorted_countries = list(salaries2["company_location"].value_counts().index)
number_of_employees = salaries2["company_location"].value_counts()
len(number_of_employees)

In [ ]:
#prueba["company_location"] = pd.Categorical(prueba["company_location"], categories = sorted_countries)
a = prueba.sort_values(by = "company_location")
#a
a

In [ ]:
b =  a.sort_values(by=['salary_in_usd'], ascending=False)
#a['company_location'] = number_of_employees.values
#a
b
super_high = ['US','NZ','CA','CA','AE','CN','DZ','SG','DK','JP','AU','BE']
high = ['DE','GB','AT','PL','IE','CZ','FR','CH','SI','RO','NL']
medium=['GR','ES','PT','LU','CL','HU','EE','IN','NG','MT']


In [ ]:
#a['mean_salary']= a['salary_in_usd']/a['employee_number']
#a

In [ ]:
number_of_employees.values

In [ ]:
#b =  a.sort_values(by=['salary_in_usd'], ascending=False)
#b['number'] = number_of_employees.values
#b

In [ ]:
#Esto no porque da una media de salario para algunos paises que no tiene sentido 
#b['mean_salary']= b['salary_in_usd']/b['number']
#b

In [ ]:
b.describe(include='all').T

In [ ]:
b

In [ ]:
new_company=[]
for e in b.salary_in_usd:
    if e>= 85501.152174 and e <= 144892.682434: 
        new_company.append('high')
    elif e>= 54945.75 and e<= 85501.152174:
        new_company.append('medium')
    elif e >= 24911.25 and e<= 54945.75:
        new_company.append('low')
    else:
        new_company.append('super_low')

In [ ]:
b.company_location= new_company

In [ ]:
b

In [ ]:
salaries3.company_location=b.company_location

In [ ]:
salaries3

In [ ]:
super_high = ['US','NZ','CA','CA','AE','CN','DZ','SG','DK','JP','AU','BE']
high = ['DE','GB','AT','PL','IE','CZ','FR','CH','SI','RO','NL']
medium=['GR','ES','PT','LU','CL','HU','EE','IN','NG','MT']

In [ ]:
new_location = []
for e in salaries3.company_location:
    if e in super_high:
        new_location.append('super_high')
    elif e in high:
        new_location.append('high')
    elif e in medium:
        new_location.append('medium')
    else:
        new_location.append('low')


        


In [ ]:
salaries3.company_location = new_location

In [ ]:
salaries3.drop('employee_residence',axis=1, inplace=True)

In [ ]:
salaries3.head()

In [ ]:
salaries3=pd.get_dummies(salaries3, columns=['company_location'], drop_first=True)

salaries3.head()

In [ ]:
salaries4 = salaries3.iloc[:500,:]
testeofin= salaries3.iloc[500:,:]

In [ ]:
testeofin.reset_index()

In [ ]:
testeofin.reset_index(inplace=True)

In [ ]:
testeofin

In [ ]:
testeofin.drop('index',axis=1,inplace=True)

In [ ]:
testeofin

In [ ]:
salaries4.head()

In [ ]:
testeofin.shape

In [ ]:
salaries4.shape

In [ ]:
#from sklearn.ensemble import RandomForestRegressor as RFR  
from sklearn.model_selection import train_test_split as tts      # el alias es mio


In [ ]:
rfr=RFR()


In [ ]:
x_train, x_test, y_train, y_test = tts(salaries4, y, train_size=0.8, test_size=0.2, random_state=22)


In [ ]:
len(y_train)

In [ ]:
rfr.fit(X_train, y_train)


In [ ]:
y_pred_rf = rfr.predict(X_test)

In [ ]:
from sklearn.metrics import mean_squared_error as mse   


In [ ]:
mse(y_test, y_pred, squared=False)

In [ ]:
rfr.fit(salaries4,y)


In [ ]:
y_pred_test = rfr.predict(testeofin)

In [ ]:
y_pred_test

In [ ]:
muestra.salary_in_usd = y_pred_test

In [ ]:
error1 = muestra.copy()

In [ ]:
error1.to_csv('../data/error1.csv', index=False)



In [ ]:
from sklearn.svm import SVR  

In [ ]:
svr=SVR()

In [ ]:
svr.fit(X_train, y_train)

In [ ]:
y_pred_svr = svr.predict(X_test)

In [ ]:
mse(y_test, y_pred, squared=False)

In [ ]:
svr.fit(salaries4,y)


In [ ]:
y_pred_svr = rfr.predict(testeofin)

In [ ]:
muestra.salary_in_usd = y_pred_svr

In [ ]:
error2 = muestra.copy()

In [ ]:
error2.to_csv('../data/error2.csv', index=False)


In [ ]:

from sklearn.preprocessing import StandardScaler

scaler=StandardScaler()

In [ ]:
salaries4[['work_year','remote_ratio']] = scaler.fit_transform(salaries4[['work_year','remote_ratio']])


In [ ]:
salaries4.head()

In [ ]:
rfr2=RFR()

In [ ]:
rfr2.fit(X_train, y_train)


In [ ]:
y_pred_rf2 = rfr2.predict(X_test)

In [ ]:
mse(y_test, y_pred2, squared=False)

In [ ]:
rfr2.fit(salaries4,y)


In [ ]:
y_pred_test2 = rfr2.predict(testeofin)

In [ ]:
muestra.salary_in_usd = y_pred_test2

In [ ]:
error3 = muestra.copy()

In [ ]:
error3.to_csv('../data/error3.csv', index=False)


In [ ]:
x_train, x_test, y_train, y_test = tts(salaries4, y, train_size=0.8, test_size=0.2, random_state=22)

In [ ]:
%pip install xgboost

In [ ]:
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor as RFR  
from sklearn.tree import ExtraTreeRegressor as ETR
from sklearn.ensemble import GradientBoostingRegressor as GBR
from xgboost import XGBRegressor as XGBR
from catboost import CatBoostRegressor as CTR
from sklearn.linear_model import LinearRegression as LinReg
from sklearn.linear_model import Lasso    
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet 

In [ ]:
def predecir(x_train, x_test, y_train, y_test):
    #inicializamos todos los modelos que vamos a probar
    svr=SVR()
    rfr=RFR()
    etr=ETR()
    gbr=GBR()
    xgbr=XGBR()
    ctr=CTR()
    linreg=LinReg()
    lasso=Lasso()
    ridge=Ridge()
    elastic=ElasticNet()
    #los entrenamos
    svr.fit(x_train, y_train)
    rfr.fit(x_train, y_train)
    etr.fit(x_train, y_train)
    gbr.fit(x_train, y_train)
    xgbr.fit(x_train, y_train)
    ctr.fit(x_train, y_train, verbose=0)
    linreg.fit(x_train, y_train)
    lasso.fit(x_train, y_train)
    ridge.fit(x_train, y_train)
    elastic.fit(x_train, y_train)
    #predecimos
    y_pred1 = svr.predict(x_test)
    y_pred2 = rfr.predict(x_test)
    y_pred3 = etr.predict(x_test)
    y_pred4 = gbr.predict(x_test)
    y_pred5 = xgbr.predict(x_test)
    y_pred6 = ctr.predict(x_test)
    y_pred7 = linreg.predict(x_test)
    y_pred8 = lasso.predict(x_test)
    y_pred9 = ridge.predict(x_test)
    y_pred10 = elastic.predict(x_test)
    #calculamos error cuadrático medio (mse)
    mse1 = mse(y_test, y_pred1, squared=False)
    mse2 = mse(y_test, y_pred2, squared=False)
    mse3 = mse(y_test, y_pred3, squared=False)
    mse4 = mse(y_test, y_pred4, squared=False)
    mse5 = mse(y_test, y_pred5, squared=False)
    mse6 = mse(y_test, y_pred6, squared=False)
    mse7 = mse(y_test, y_pred7, squared=False)
    mse8 = mse(y_test, y_pred8, squared=False)
    mse9 = mse(y_test, y_pred9, squared=False)
    mse10 = mse(y_test, y_pred10, squared=False)
    #creamos una lista con todos los mse
    temp = [mse1, mse2, mse3, mse4, mse5, mse6, mse7, mse8, mse9, mse10]
    #pedimos a la función que nos devuelva el valor más bajo de mse
    minimo = min(temp)
    #le ponemos un mensajito para que quede más mono
    if minimo == mse1:
        return f'El mejor modelo es SVR con un mse de {mse1}'
    elif minimo == mse2:
        return f'El mejor modelo es RFR con un mse de {mse2}'
    elif minimo == mse3:
        return f'El mejor modelo es ETR con un mse de {mse3}'
    elif minimo == mse4:
        return f'El mejor modelo es GBR con un mse de {mse4}'
    elif minimo == mse5:
        return f'El mejor modelo es XGBR con un mse de {mse5}'
    elif minimo == mse6:
        return f'El mejor modelo es CTR con un mse de {mse6}'
    elif minimo == mse7:
        return f'El mejor modelo es LINREG con un mse de {mse7}'
    elif minimo == mse8:
        return f'El mejor modelo es LASSO con un mse de {mse8}'
    elif minimo == mse9:
        return f'El mejor modelo es RIDGE con un mse de {mse9}'
    elif minimo == mse10:
        return f'El mejor modelo es ELASTIC con un mse de {mse10}'

In [ ]:
predecir(x_train, x_test, y_train, y_test)

In [ ]:
rfr=RFR()

In [ ]:
rfr.fit(x_train, y_train)

In [ ]:
y_pred_rf3 = rfr.predict(x_test)

In [ ]:
mse14 = mse(y_test, y_pred_rf3, squared=False)

In [ ]:
mse14

In [ ]:
mse8 = mse(y_test, y_pred8, squared=False)

In [ ]:
mse8

In [ ]:
y_pred_test5 = rfr.predict(testeofin)

In [ ]:
muestra.salary_in_usd = y_pred_test5

In [ ]:
error5 = muestra.copy()

In [ ]:
error5.to_csv('../data/error5.csv', index=False)
